# Sanfrancisco Test Vis
##Smaller heading

#

I would like to investigate the correlation between the time of day and the number of police incidents which were recorded  during the summer of 2014 in San Francisco. I suspect that there would be more incidents during the evening than the day. 

To see if this suspicion is true and if there are any other correlations we created this Jupyter notebook. Initially we load the San Francisco incidents dataset using pandas. We need to convert the date columns into a datetime column so that we can query it. We also split the time column into an hour and minute column to help quering.

In [2]:
import pandas as pd
df_SF = pd.read_csv('sanfrancisco_incidents_summer_2014.csv', sep=",")
df_SF['Date'] =  pd.to_datetime(df_SF['Date'], format='%m/%d/%Y')
df_SF['Hour'] = pd.to_datetime(df_SF['Time'], format='%H:%M').dt.hour
df_SF['Minute'] = pd.to_datetime(df_SF['Time'], format='%H:%M').dt.minute

HourGroup_SF = df_SF.groupby(pd.Grouper(key="Hour")).count().reset_index()

# Load the Seattle data set also
df_SEA = pd.read_csv('seattle_incidents_summer_2014.csv', sep=",")
df_SEA['Hour'] =  pd.to_datetime(df_SEA['Date Reported']).dt.hour

HourGroup_SEA = df_SEA.groupby(pd.Grouper(key="Hour", sort="True")).count().reset_index()

We also would like the find the hour in the day with the most and least number of occurrences for each city.

In [33]:
minHour_SF = HourGroup_SF.loc[HourGroup_SF['IncidntNum'].idxmin()].Hour
maxHour_SF = HourGroup_SF.loc[HourGroup_SF['IncidntNum'].idxmax()].Hour
minHour_SEA = HourGroup_SEA.loc[HourGroup_SEA['General Offense Number'].idxmin()].Hour
maxHour_SEA = HourGroup_SEA.loc[HourGroup_SEA['General Offense Number'].idxmax()].Hour
print "San Francisco - min :" + str(minHour_SF) + " max : " + str(maxHour_SF)
print "Seattle - min : " + str(minHour_SEA) + " max : " + str(maxHour_SEA)

SF - min :4 max : 18
SEA - min : 5 max : 13


In [111]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.models import PrintfTickFormatter, Span, ColumnDataSource
from bokeh.palettes import Category20
from bokeh.transform import cumsum
from math import pi

output_notebook()

p = figure(title="Total number of police incidents at each hour throughout the summer of 2014")
p.line(x=HourGroup_SEA["Hour"],y=HourGroup_SEA["General Offense Number"], line_width=6, line_color="Blue", legend="Seattle")
p.line(x=HourGroup_SF["Hour"],y=HourGroup_SF["IncidntNum"], line_width=6, line_color="Green", legend="SF")

SF_MIN = Span(location=minHour_SF,dimension='height', line_color='Green',line_width=2)
SF_MAX = Span(location=maxHour_SF,dimension='height', line_color='Green',line_width=2)
SEA_MIN = Span(location=minHour_SEA,dimension='height', line_color='Blue',line_width=2)
SEA_MAX = Span(location=maxHour_SEA,dimension='height', line_color='Blue',line_width=2)
p.renderers.extend([SF_MIN, SF_MAX, SEA_MIN, SEA_MAX])

p.yaxis.axis_label = "Number of Incidents"
p.xaxis.axis_label = "Time of Incident"
p.xaxis[0].formatter = PrintfTickFormatter(format="%d:00")

show(p)

Loading BokehJS ...

From the graph above it can be seen that in San Francisco the peak time for police incidents is 18:00 while the lowest 
number of incidents occur at 4:00. This is as I suspected that more crimes occur in the evening than during the day (which can also be seen by the sinusodal shape of the graph. However, what is unexpected is that the peak occurs so early 18:00 , when it is still light during the summer months. 

Even more suprisingly in Seattle where the peak time is 13:00 which is much earlier than my initial suspicions. 

In [92]:
print len(df_SF['Descript'].unique())

df_SFDescriptGROUP = df_SF.groupby(pd.Grouper(key="Descript")).count().reset_index()
df_SFDescriptGROUP = df_SFDescriptGROUP.sort_values(by=["IncidntNum"], ascending=False)


368


In [112]:
# prepare data for our graphs - 
listOfCategories = ["ROBBERY","THEFT", "ASSAULT", "POSSESSION", "SALE", "HOMICIDE", "BATTERY", "LOST", "STOLEN" ];
data = {
    "categories": listOfCategories,
    "min":[],
    "max":[],
    "totIncidents":[]
}

def extractDetails(df):
    data['totIncidents'].append(len(df))
    df_Hour = df.groupby(pd.Grouper(key="Hour")).count().reset_index()
    data['max'].append(df_Hour.loc[df_Hour['IncidntNum'].idxmax()].Hour)
    data['min'].append(df_Hour.loc[df_Hour['IncidntNum'].idxmax()].Hour)

for c in data["categories"]:
    df_SF_Filtered = df_SF[df_SF.apply(lambda x: c in x['Descript'], axis=1, result_type='reduce')]
    extractDetails(df_SF_Filtered)

# Get all entries which don't fit in any category   
df_SF_Filtered_Other = df_SF[df_SF.apply(lambda x: True if not any(s in x['Descript'] for s in listOfCategories) else False, 
                                         axis=1, 
                                         result_type='reduce')]
data['categories'].append("OTHER")
extractDetails(df_SF_Filtered_Other)

allIncidents = float(sum(data['totIncidents']))
data['percent'] = map(lambda x: '{0:.2f}'.format(x/allIncidents*100.0) + "%", data['totIncidents'])
data['angle'] = map(lambda x: x/allIncidents*2*pi, data['totIncidents'])
data['color'] = Category20[len(data['categories'])]

# print data

p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
        tools="hover", tooltips="@categories: @percent")

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='categories', source=data)

show(p)

{'max': [23, 18, 2, 17, 11, 22, 17, 12, 18, 12], 'totIncidents': [300, 9516, 746, 1361, 359, 30, 1035, 1518, 2290, 12395], 'categories': ['ROBBERY', 'THEFT', 'ASSAULT', 'POSSESSION', 'SALE', 'HOMICIDE', 'BATTERY', 'LOST', 'STOLEN', 'OTHER'], 'min': [10, 4, 7, 4, 5, 1, 6, 4, 4, 4]}
